# Chapter 16

# Exercise 8

Embedded Reber grammars were used by Hochreiter and Schmidhuber in their paper about LSTMs. They are artificial grammars that produce strings such as “BPBTSXXVPSEPE.” Check out Jenny Orr’s nice introduction to this topic. Choose a particular embedded Reber grammar (such as the one represented on Jenny Orr’s page), then train an RNN to identify whether a string respects that grammar or not. You will first need to write a function capable of generating a training batch containing about 50% strings that respect the grammar, and 50% that don’t.

In [126]:
from collections import defaultdict
from random import choice, sample

In [147]:
%run ./reber.py

## Reber

In [128]:
reber_edges = ((0,1,'B'), (1,2,'T'), (1,3,'P'), (2,2,'S'), (2,4,'X'), (3,3,'T'), (3,5,'V'), (4,3,'X'), (4,6,'S'), (5,4,'P'), (5,6,'V'), (6,None,'E'))

In [129]:
node_dict = dict_from_edges(reber_edges)

In [130]:
node_dict

defaultdict(list,
            {0: [(1, 'B')],
             1: [(2, 'T'), (3, 'P')],
             2: [(2, 'S'), (4, 'X')],
             3: [(3, 'T'), (5, 'V')],
             4: [(3, 'X'), (6, 'S')],
             5: [(4, 'P'), (6, 'V')],
             6: [(None, 'E')]})

In [131]:
sentence = generate_sentence(node_dict)

In [132]:
sentence

((0, 1, 'B'),
 (1, 3, 'P'),
 (3, 3, 'T'),
 (3, 5, 'V'),
 (5, 4, 'P'),
 (4, 3, 'X'),
 (3, 5, 'V'),
 (5, 6, 'V'),
 (6, None, 'E'))

In [133]:
string_from_sentence(sentence)

'BPTVPXVVE'

In [134]:
unique_letters(sentence)

'PEBTVX'

In [135]:
unique_letters(reber_edges)

'PEBTSVX'

In [136]:
sentence_edge = sentence[3]

In [137]:
sentence_edge

(3, 5, 'V')

In [138]:
corrupted_sentence_edge = corrupt_edge(sentence_edge, edges)

In [139]:
corrupted_sentence_edge

(3, 5, 'B')

In [140]:
corrupted_sentence = corrupt_sentence(sentence, reber_edges, 2)

In [142]:
corrupted_sentence

((0, 1, 'B'),
 (1, 3, 'P'),
 (3, 3, 'T'),
 (3, 5, 'V'),
 (5, 4, 'P'),
 (4, 3, 'B'),
 (3, 5, 'V'),
 (5, 6, 'T'),
 (6, None, 'E'))

## Embedder Reber Grammar

In [148]:
embedded_reber_edges = ((0,1,'B'), (1,2,'T'), (1,3,'P'), (2,4,reber_edges), (3,5,reber_edges), (4,6, 'T'), (5,6,'P'), (6,None,'E'))

In [149]:
embedded_reber_edges = flatten_embedded_edges(embedded_reber_edges)

In [150]:
embedded_reber_edges

((0, 1, 'B'),
 (1, 2, 'T'),
 (1, 3, 'P'),
 (2, '2-1', 'B'),
 ('2-1', '2-2', 'T'),
 ('2-1', '2-3', 'P'),
 ('2-2', '2-2', 'S'),
 ('2-2', '2-4', 'X'),
 ('2-3', '2-3', 'T'),
 ('2-3', '2-5', 'V'),
 ('2-4', '2-3', 'X'),
 ('2-4', '2-6', 'S'),
 ('2-5', '2-4', 'P'),
 ('2-5', '2-6', 'V'),
 ('2-6', 4, 'E'),
 (3, '3-1', 'B'),
 ('3-1', '3-2', 'T'),
 ('3-1', '3-3', 'P'),
 ('3-2', '3-2', 'S'),
 ('3-2', '3-4', 'X'),
 ('3-3', '3-3', 'T'),
 ('3-3', '3-5', 'V'),
 ('3-4', '3-3', 'X'),
 ('3-4', '3-6', 'S'),
 ('3-5', '3-4', 'P'),
 ('3-5', '3-6', 'V'),
 ('3-6', 5, 'E'),
 (4, 6, 'T'),
 (5, 6, 'P'),
 (6, None, 'E'))

In [151]:
node_dict = dict_from_edges(embedded_reber_edges)

In [152]:
node_dict

defaultdict(list,
            {0: [(1, 'B')],
             1: [(2, 'T'), (3, 'P')],
             2: [('2-1', 'B')],
             '2-1': [('2-2', 'T'), ('2-3', 'P')],
             '2-2': [('2-2', 'S'), ('2-4', 'X')],
             '2-3': [('2-3', 'T'), ('2-5', 'V')],
             '2-4': [('2-3', 'X'), ('2-6', 'S')],
             '2-5': [('2-4', 'P'), ('2-6', 'V')],
             '2-6': [(4, 'E')],
             3: [('3-1', 'B')],
             '3-1': [('3-2', 'T'), ('3-3', 'P')],
             '3-2': [('3-2', 'S'), ('3-4', 'X')],
             '3-3': [('3-3', 'T'), ('3-5', 'V')],
             '3-4': [('3-3', 'X'), ('3-6', 'S')],
             '3-5': [('3-4', 'P'), ('3-6', 'V')],
             '3-6': [(5, 'E')],
             4: [(6, 'T')],
             5: [(6, 'P')],
             6: [(None, 'E')]})

In [153]:
sentence = generate_sentence(node_dict)

In [154]:
sentence

((0, 1, 'B'),
 (1, 2, 'T'),
 (2, '2-1', 'B'),
 ('2-1', '2-2', 'T'),
 ('2-2', '2-4', 'X'),
 ('2-4', '2-3', 'X'),
 ('2-3', '2-3', 'T'),
 ('2-3', '2-3', 'T'),
 ('2-3', '2-5', 'V'),
 ('2-5', '2-6', 'V'),
 ('2-6', 4, 'E'),
 (4, 6, 'T'),
 (6, None, 'E'))

In [155]:
string_from_sentence(sentence)

'BTBTXXTTVVETE'

In [156]:
corrupt_sentence(sentence, embedded_reber_edges, 3)

((0, 1, 'B'),
 (1, 2, 'T'),
 (2, '2-1', 'B'),
 ('2-1', '2-2', 'S'),
 ('2-2', '2-4', 'X'),
 ('2-4', '2-3', 'X'),
 ('2-3', '2-3', 'T'),
 ('2-3', '2-3', 'B'),
 ('2-3', '2-5', 'V'),
 ('2-5', '2-6', 'V'),
 ('2-6', 4, 'E'),
 (4, 6, 'S'),
 (6, None, 'E'))

## Generate Training Data

In [157]:
n_positive = 10000
max_corruptions = 3

In [40]:
def forward_dict_from_edges(edges):
    forward_dict = defaultdict(list)
    for (start_node, end_node, letter) in edges:
        forward_dict[start_node].append((end_node, letter))
    return forward_dict

In [41]:
forward_dict = forward_dict_from_edges(reber_edges)

In [42]:
forward_dict

defaultdict(list,
            {0: [(1, 'B')],
             1: [(2, 'T'), (3, 'P')],
             2: [(2, 'S'), (4, 'X')],
             3: [(3, 'T'), (5, 'V')],
             4: [(3, 'X'), (6, 'S')],
             5: [(4, 'P'), (6, 'V')],
             6: [(None, 'E')]})

In [43]:
def generate_sentence(forward_dict):
	sentence = []
	start_node = 0
	while start_node is not None:
		end_node, letter = choice(forward_dict[start_node])
		sentence.append([start_node, end_node, letter])
		start_node = end_node
	return sentence

In [44]:
sentence = generate_sentence(forward_dict)

In [45]:
sentence

[[0, 1, 'B'],
 [1, 2, 'T'],
 [2, 2, 'S'],
 [2, 4, 'X'],
 [4, 3, 'X'],
 [3, 5, 'V'],
 [5, 4, 'P'],
 [4, 3, 'X'],
 [3, 5, 'V'],
 [5, 6, 'V'],
 [6, None, 'E']]

In [46]:
def string_from_sentence(sentence):
    return ''.join([edge[-1] for edge in sentence])

In [47]:
string_from_sentence(sentence)

'BTSXXVPXVVE'

In [48]:
def letters_from_edges(edges):
    return ''.join(set([edge[-1] for edge in edges]))

In [49]:
chars = letters_from_edges(reber_edges)

In [50]:
chars

'PEBTSVX'

In [60]:
def corrupt_letter(sentence_edge, edges):
    start_node = sentence_edge[0]
    possible_nodes = [edge for edge in edges if edge[0] == start_node]
    possible_letters = [node[-1] for node in possible_nodes]
    all_letters = letters_from_edges(edges)
    corruption_letters = list(set(all_letters) - set(possible_letters))
    new_letter = choice(corruption_letters)
    corrupted_sentence_edge = sentence_edge.copy()
    corrupted_sentence_edge[-1] = new_letter
    return corrupted_sentence_edge

In [61]:
sentence

[[0, 1, 'B'],
 [1, 2, 'T'],
 [2, 2, 'S'],
 [2, 4, 'X'],
 [4, 3, 'X'],
 [3, 5, 'V'],
 [5, 4, 'P'],
 [4, 3, 'X'],
 [3, 5, 'V'],
 [5, 6, 'V'],
 [6, None, 'E']]

In [64]:
sentence_edge = sentence[3]

In [65]:
sentence_edge

[2, 4, 'X']

In [63]:
corrupt_letter(sentence_edge, reber_edges)

[2, 4, 'T']

In [72]:
def corrupt_sentence(sentence, edges, num_corruptions):
    assert num_corruptions <= len(sentence)
    index_corruptions = sample(range(len(sentence)), num_corruptions)
    corrupted_sentence = sentence.copy()
    for index in index_corruptions:
        corrupted_sentence[index] = corrupt_letter(corrupted_sentence[index], edges)
    return corrupted_sentence

In [73]:
sentence

[[0, 1, 'B'],
 [1, 2, 'T'],
 [2, 2, 'S'],
 [2, 4, 'X'],
 [4, 3, 'X'],
 [3, 5, 'V'],
 [5, 4, 'P'],
 [4, 3, 'X'],
 [3, 5, 'V'],
 [5, 6, 'V'],
 [6, None, 'E']]

In [74]:
num_corruptions = 3

In [76]:
corrupt_sentence(sentence, reber_edges, num_corruptions)

[[0, 1, 'S'],
 [1, 2, 'T'],
 [2, 2, 'S'],
 [2, 4, 'X'],
 [4, 3, 'X'],
 [3, 5, 'B'],
 [5, 4, 'P'],
 [4, 3, 'P'],
 [3, 5, 'V'],
 [5, 6, 'V'],
 [6, None, 'E']]

## Embedded Reber Grammar

In [110]:
start_node, end_node, letter = embedded_reber_edges[3]

In [115]:
t = [[str(start_node) + '-' + str(s), str(start_node)+'-'+str(e), l] for s, e, l in letter]

In [118]:
t[-1][1] = end_node

In [119]:
t

[['2-0', '2-1', 'B'],
 ['2-1', '2-2', 'T'],
 ['2-1', '2-3', 'P'],
 ['2-2', '2-2', 'S'],
 ['2-2', '2-4', 'X'],
 ['2-3', '2-3', 'T'],
 ['2-3', '2-5', 'V'],
 ['2-4', '2-3', 'X'],
 ['2-4', '2-6', 'S'],
 ['2-5', '2-4', 'P'],
 ['2-5', '2-6', 'V'],
 ['2-6', 4, 'E']]

In [125]:
new_edges

[[0, 1, 'B'],
 [1, 2, 'T'],
 [1, 3, 'P'],
 ['2-0', '2-1', 'B'],
 ['2-1', '2-2', 'T'],
 ['2-1', '2-3', 'P'],
 ['2-2', '2-2', 'S'],
 ['2-2', '2-4', 'X'],
 ['2-3', '2-3', 'T'],
 ['2-3', '2-5', 'V'],
 ['2-4', '2-3', 'X'],
 ['2-4', '2-6', 'S'],
 ['2-5', '2-4', 'P'],
 ['2-5', '2-6', 'V'],
 ['2-6', 4, 'E'],
 ['3-0', '3-1', 'B'],
 ['3-1', '3-2', 'T'],
 ['3-1', '3-3', 'P'],
 ['3-2', '3-2', 'S'],
 ['3-2', '3-4', 'X'],
 ['3-3', '3-3', 'T'],
 ['3-3', '3-5', 'V'],
 ['3-4', '3-3', 'X'],
 ['3-4', '3-6', 'S'],
 ['3-5', '3-4', 'P'],
 ['3-5', '3-6', 'V'],
 ['3-6', 5, 'E'],
 [4, 6, 'T'],
 [5, 6, 'P'],
 [6, None, 'E']]

In [86]:
def forward_dict_from_edges(edges):
    forward_dict = defaultdict(list)
    for (start_node, end_node, letter) in edges:
        if isinstance(letter, tuple):
            letter = forward_dict_from_edges(letter)
        forward_dict[start_node].append((end_node, letter))
    return forward_dict

In [81]:
reber_edges

((0, 1, 'B'),
 (1, 2, 'T'),
 (1, 3, 'P'),
 (2, 2, 'S'),
 (2, 4, 'X'),
 (3, 3, 'T'),
 (3, 5, 'V'),
 (4, 3, 'X'),
 (4, 6, 'S'),
 (5, 4, 'P'),
 (5, 6, 'V'),
 (6, None, 'E'))

In [87]:
forward_dict_from_edges(reber_edges)

defaultdict(list,
            {0: [(1, 'B')],
             1: [(2, 'T'), (3, 'P')],
             2: [(2, 'S'), (4, 'X')],
             3: [(3, 'T'), (5, 'V')],
             4: [(3, 'X'), (6, 'S')],
             5: [(4, 'P'), (6, 'V')],
             6: [(None, 'E')]})

In [101]:
forward_dict = forward_dict_from_edges(embedded_reber_edges)

In [102]:
forward_dict

defaultdict(list,
            {0: [(1, 'B')],
             1: [(2, 'T'), (3, 'P')],
             2: [(4,
               defaultdict(list,
                           {0: [(1, 'B')],
                            1: [(2, 'T'), (3, 'P')],
                            2: [(2, 'S'), (4, 'X')],
                            3: [(3, 'T'), (5, 'V')],
                            4: [(3, 'X'), (6, 'S')],
                            5: [(4, 'P'), (6, 'V')],
                            6: [(None, 'E')]}))],
             3: [(5,
               defaultdict(list,
                           {0: [(1, 'B')],
                            1: [(2, 'T'), (3, 'P')],
                            2: [(2, 'S'), (4, 'X')],
                            3: [(3, 'T'), (5, 'V')],
                            4: [(3, 'X'), (6, 'S')],
                            5: [(4, 'P'), (6, 'V')],
                            6: [(None, 'E')]}))],
             4: [(6, 'T')],
             5: [(6, 'P')],
             6: [(None, 'E')]})

In [94]:
def generate_sentence(forward_dict):
    sentence = []
    start_node = 0
    while start_node is not None:
        end_node, letter = choice(forward_dict[start_node])
        if isinstance(letter, defaultdict):
            letter = generate_sentence(letter)
        sentence.append([start_node, end_node, letter])
        start_node = end_node
    return sentence

In [98]:
sentence = generate_sentence(forward_dict)

In [99]:
def string_from_sentence(sentence):
    letters = []
    for edge in sentence:
        letter = edge[-1]
        if isinstance(letter, list):
            letter = string_from_sentence(letter)
        letters.append(letter)
    return ''.join(letters)

In [103]:
string_from_sentence(sentence)

'BTBPTTVPSETE'

In [ ]:
def corrupt_letter(sentence_edge, edges):
    start_node = sentence_edge[0]
    possible_nodes = []
    for edge in edges:
        
    possible_nodes = [edge for edge in edges if edge[0] == start_node]
    possible_letters = [node[-1] for node in possible_nodes]
    all_letters = letters_from_edges(edges)
    corruption_letters = list(set(all_letters) - set(possible_letters))
    new_letter = choice(corruption_letters)
    corrupted_sentence_edge = sentence_edge.copy()
    corrupted_sentence_edge[-1] = new_letter
    return corrupted_sentence_edge